# LGBM Prediction



In [1]:
from google.colab import drive
drive.mount('/content/drive')
base_path = '/content/drive/MyDrive/RecSys2024/'
# 3.6.0以降だとLightGBMTunerが動かない
#!pip3 install optuna==3.5.0
!pip3 install polars lightgbm pyarrow

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
#DEBUG_MODE = True
DEBUG_MODE = False
TRAIN_FRAC = 1.0 # train dataのnegative sampleをどれくらいsamplingするか
#TRAIN_FRAC = 0.8 # train dataのnegative sampleをどれくらいsamplingするか
#TRAIN_FRAC = 0.5 # train dataのnegative sampleをどれくらいsamplingするか
VALID_FRAC = 0.25 # validatoin dataをどれくらいsamplingするか
#lgbm_valid_frac1.0.pkl

train_type = 'train'
#train_type = 'valid' # use validation data as training for final su

In [3]:
# ==================================================== # Library # ====================================================
import os
import gc
import warnings
warnings.filterwarnings('ignore')
import random
import scipy as sp
import numpy as np
import pandas as pd
import polars as pl
import joblib
import pyarrow
import itertools
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
from tqdm.auto import tqdm
from sklearn.model_selection import KFold, train_test_split
from sklearn.metrics import accuracy_score, log_loss
import lightgbm as lgb
print(lgb.__version__)

4.3.0


# Reading Input File

In [4]:
N_split = 2 # N個に分割して推論
out_path = base_path + 'feature_output'
model_path = base_path + 'models'
TRAIN_FRAC = '1.0'
bins = [600, 3600, 3600 * 6]

# Feature Engineering

In [5]:
def read_files(i_chunk, N_split=2):
    #impression = pl.read_parquet(f"{out_path}/test_impression.parquet").sort(["impression_id", "user_id"])
    #impression_article = pl.read_parquet(f"{out_path}/test_impression_article.parquet").sort(["impression_id", "user_id"])
    impression = pl.read_parquet(f"{out_path}/test_impression.parquet")
    impression_article = pl.read_parquet(f"{out_path}/test_impression_article.parquet")
    user = pl.read_parquet(f"{out_path}/test_user.parquet")
    user_article = pl.read_parquet(f"{out_path}/test_user_article.parquet")
    article = pl.read_parquet(f"{out_path}/test_article.parquet")

    rows_per_chunk = len(impression) // N_split
    chunks = [impression.slice(i * rows_per_chunk, rows_per_chunk) for i in range(N_split)]
    impression = chunks[i_chunk]
    del chunks, rows_per_chunk
    gc.collect()
    return impression_article, impression, user, user_article, article

def add_pop_count(df, out_col_name, bins):

    for bin in bins:
        df = df.with_columns(
            (df["impression_time"] / bin).cast(pl.Int64).alias(f"time_bin_{bin}")
        )
        grouped_df = df.group_by([f"time_bin_{bin}", "article_id_inview"]).agg(
            pl.count().cast(pl.Int32).alias(out_col_name + str(bin))
        )
        df = df.join(
            grouped_df,
            on=[f"time_bin_{bin}", "article_id_inview"],
            how="left"
        )
        df = df.drop(f"time_bin_{bin}")
    return df


def join_features(impression_article, impression, user, user_article, article, bins):
    print('join features....')
    df = impression_article.join(impression, on=["impression_id", "user_id"], how="inner")
    del impression_article
    gc.collect()
    df = reduce_mem_usage(df)

    print('add view_cnt....')
    df = add_pop_count(df,"view_cnt", bins)
    df = reduce_mem_usage(df)

    print('add user features....')
    df = df.join(user, on="user_id", how="left")
    del user
    gc.collect()
    df = reduce_mem_usage(df)

    print('add article features....')
    df = df.rename({
        "article_id_inview": "to_article_id"
    })
    df = df.join(
        article.rename({col: f"to_{col}" for col in article.columns}),
        on="to_article_id",
        how="left"
    )
    del article
    gc.collect()
    df = reduce_mem_usage(df)

    print('add user article features....')
    df = df.join(
        user_article.rename({col: f"to_{col}" for col in user_article.columns if col != 'user_id'}),
        on=["user_id", "to_article_id"],
        how="left"
    )
    del user_article
    gc.collect()
    df = reduce_mem_usage(df)

    print('cast features....')
    for bin in bins:
        df = df.with_columns(
            pl.col(f"view_cnt{bin}").fill_null(0).cast(pl.Int32)
        )
    df = reduce_mem_usage(df)
    return df


def generate_unixtime_features(df, unixtime_list):

    for col_name in unixtime_list:
        df = df.with_columns(
            (pl.col("impression_time") - pl.col(col_name)).alias(f"{col_name}_diff").cast(pl.Int64)
        )
        df = df.drop(col_name)
    df = df.drop('impression_time')
    df = reduce_mem_usage(df)
    return df


def to_pandas(df_data, cat_cols=None):
    df_data = df_data.to_pandas()
    if cat_cols is None:
        cat_cols = list(df_data.select_dtypes("object").columns)
    df_data[cat_cols] = df_data[cat_cols].astype("category")
    return df_data
    #return df_data, cat_cols


def reduce_mem_usage(df):
    """ iterate through all the columns of a dataframe and modify the data type
        to reduce memory usage in Polars.
    """

    # Initialize an empty list to store optimized columns
    optimized_columns = []

    for col in df.columns:
        col_data = df[col]
        col_type = col_data.dtype

        if col_type in [pl.Int8, pl.Int16, pl.Int32, pl.Int64, pl.UInt32]:
            col_data = col_data.fill_null(0)
            c_min = col_data.min()
            c_max = col_data.max()
            if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                optimized_columns.append(col_data.cast(pl.Int8))
            elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                optimized_columns.append(col_data.cast(pl.Int16))
            elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                optimized_columns.append(col_data.cast(pl.Int32))
            else:
                optimized_columns.append(col_data.cast(pl.Int64))
        elif col_type in [pl.Float32, pl.Float64]:
            c_min = col_data.min()
            c_max = col_data.max()
            if c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                optimized_columns.append(col_data.cast(pl.Float32))
            else:
                optimized_columns.append(col_data.cast(pl.Float64))
        else:
            optimized_columns.append(col_data)
    # Create a new DataFrame with optimized columns
    optimized_df = pl.DataFrame({col: optimized_columns[i] for i, col in enumerate(df.columns)})

    return optimized_df


def cast_float_to_int(df, columns):
    for col in columns:
        df = df.with_columns(pl.col(col).cast(pl.Int64))
    return df


def negative_sampling(df, frac=0.5, seed=42):
    negative_examples = df.filter(pl.col("clicked") == False)
    positive_examples = df.filter(pl.col("clicked") == True)
    reduced_negatives = negative_examples.sample(fraction = frac, seed=seed)

    return pl.concat([reduced_negatives, positive_examples]).sort("impression_id")

In [6]:
# is_beyond_accuracyはtest dataだけ入ってるので削る
IGNORE_COL_ID = ["impression_id", "session_id", "is_beyond_accuracy", "user_id", "to_article_id"]

model = np.load(f'{model_path}/lgbm_{train_type}_frac{TRAIN_FRAC}.pkl', allow_pickle=True)
model

In [7]:
sub_list = []

for i in range(N_split):
    print(f'chunk {i}')
    test_impression_article, test_impression, test_user, test_user_article, test_article = read_files(i_chunk = i, N_split=N_split)
    # memory節約, intだけどfloatになってるはいったんint64にしてから変換
    float_int_cols = ['read_time', 'scroll_percentage']
    test_impression = cast_float_to_int(test_impression, float_int_cols)
    test_impression = reduce_mem_usage(test_impression)
    test_impression_article = reduce_mem_usage(test_impression_article)

    float_int_cols = ['u_read_time_last', 'u_scroll_percentage_last', 'u_impression_time_mean']
    test_user = cast_float_to_int(test_user, float_int_cols)
    test_user = reduce_mem_usage(test_user)

    float_int_cols = ['ua_read_time_last', 'ua_scroll_percentage_last']
    test_user_article = cast_float_to_int(test_user_article, float_int_cols)
    test_user_article = reduce_mem_usage(test_user_article)

    float_int_cols = ['a_total_read_time']
    test_article = cast_float_to_int(test_article, float_int_cols)
    test_article = reduce_mem_usage(test_article)

    test = join_features(test_impression_article, test_impression, test_user, test_user_article, test_article, bins)
    del test_impression_article, test_impression, test_user, test_user_article, test_article
    gc.collect()

    # unixtimeのfeatureをdiffにする
    unixtime_list = ['u_impression_time_last', 'u_impression_time_mean', 'to_ua_impression_time_mean', 'to_ua_impression_time_last', 'to_a_published_time', 'to_a_last_modified_time']
    test = generate_unixtime_features(test, unixtime_list)

    display(test.shape)
    display(test.head())
    test = to_pandas(test)

    #query_list_test = test['impression_id'].value_counts()
    #query_list_test = query_list_test.sort_index()
    print('remove id from features....')
    test_id = test[['impression_id', 'user_id', 'to_article_id']]
    test = test.drop(IGNORE_COL_ID, axis=1)
    pred = model.predict(test)

    test_id = test_id.reset_index(drop=True)
    pred = pd.Series(pred, name='score')
    sub = pd.concat([test_id, pred], axis=1)
    sub_list.append(sub)

    del test_id, pred, sub
    gc.collect()


chunk 0
join features....
add view_cnt....
add user features....
add article features....
add user article features....
cast features....


(79043265, 46)

impression_id,user_id,to_article_id,read_time,scroll_percentage,device_type,is_sso_user,gender,postcode,age,is_subscriber,session_id,is_beyond_accuracy,view_num,view_cnt600,view_cnt3600,view_cnt21600,u_history_len,u_read_time_last,u_read_time_mean,u_scroll_percentage_last,u_scroll_percentage_mean,to_a_premium,to_a_article_type,to_a_category,to_a_category_str,to_a_total_inviews,to_a_total_pageviews,to_a_total_read_time,to_a_sentiment_score,to_a_sentiment_label,to_a_ner_clusters_first,to_a_entity_groups_first,to_a_topics_first,to_a_subcategory_first,to_ua_scroll_percentage_mean,to_ua_scroll_percentage_last,to_ua_read_time_mean,to_ua_read_time_last,to_ua_count,u_impression_time_last_diff,u_impression_time_mean_diff,to_ua_impression_time_mean_diff,to_ua_impression_time_last_diff,to_a_published_time_diff,to_a_last_modified_time_diff
i32,i32,i32,i16,i8,i8,bool,i8,i8,i8,bool,i32,bool,u8,i16,i32,i32,i16,i16,f32,i8,f32,bool,str,i16,str,i32,i32,i32,f32,str,str,str,str,i16,f32,i8,f32,i16,i16,i32,i32,i32,i32,i32,i32
6451339,35982,9796527,8,0,2,false,0,0,0,false,388,false,9,1540,7849,36825,200,116,55.965,100,75.919357,true,"""article_standard_feature""",414,"""underholdning""",1674245,95347,3372986,0.7466,"""Neutral""","""Aarhus""","""LOC""","""Livsstil""",437,null,0,null,0,0,410423,1250701,0,1685977369,88394,-2043991
6451339,35982,7851321,8,0,2,false,0,0,0,false,388,false,9,1725,8504,24072,200,116,55.965,100,75.919357,true,"""article_default""",457,"""forbrug""",0,0,0,0.7398,"""Neutral""",null,null,"""Livsstil""",475,null,0,null,0,0,410423,1250701,0,1685977369,113110987,-2042686
6451339,35982,9798805,8,0,2,false,0,0,0,false,388,false,9,1774,7831,52617,200,116,55.965,100,75.919357,false,"""article_default""",142,"""sport""",340797,64344,4159228,0.4993,"""Positive""","""Barca""","""LOC""","""Erhverv""",196,null,0,null,0,0,410423,1250701,0,1685977369,10189,-2043993
6451339,35982,9795150,8,0,2,false,0,0,0,false,388,false,9,15,134,808,200,116,55.965,100,75.919357,false,"""article_default""",498,"""musik""",560052,116804,4855463,0.9576,"""Negative""","""Echo""","""LOC""","""Kendt""",504,null,0,null,0,0,410423,1250701,0,1685977369,164005,-2043989
6451339,35982,9531110,8,0,2,false,0,0,0,false,388,false,9,518,2248,12566,200,116,55.965,100,75.919357,true,"""article_default""",572,"""side9""",0,0,0,0.5158,"""Neutral""","""Cph""","""LOC""","""Livsstil""",573,null,0,null,0,0,410423,1250701,0,1685977369,16221769,-2043761


remove id from features....
chunk 1
join features....
add view_cnt....
add user features....
add article features....
add user article features....
cast features....


(126882603, 46)

impression_id,user_id,to_article_id,read_time,scroll_percentage,device_type,is_sso_user,gender,postcode,age,is_subscriber,session_id,is_beyond_accuracy,view_num,view_cnt600,view_cnt3600,view_cnt21600,u_history_len,u_read_time_last,u_read_time_mean,u_scroll_percentage_last,u_scroll_percentage_mean,to_a_premium,to_a_article_type,to_a_category,to_a_category_str,to_a_total_inviews,to_a_total_pageviews,to_a_total_read_time,to_a_sentiment_score,to_a_sentiment_label,to_a_ner_clusters_first,to_a_entity_groups_first,to_a_topics_first,to_a_subcategory_first,to_ua_scroll_percentage_mean,to_ua_scroll_percentage_last,to_ua_read_time_mean,to_ua_read_time_last,to_ua_count,u_impression_time_last_diff,u_impression_time_mean_diff,to_ua_impression_time_mean_diff,to_ua_impression_time_last_diff,to_a_published_time_diff,to_a_last_modified_time_diff
i32,i32,i32,i16,i8,i8,bool,i8,i8,i8,bool,i32,bool,u8,i32,i32,i32,i16,i16,f32,i8,f32,bool,str,i16,str,i32,i32,i32,f32,str,str,str,str,i16,f32,i8,f32,i16,i16,i32,i32,i32,i32,i32,i32
284044691,1518744,9799773,7,0,1,false,0,0,0,false,41620968,false,10,2530,14196,75134,112,9,48.767857,55,76.590477,false,"""article_scribblelive""",142,"""sport""",611193,156021,13761650,0.9582,"""Negative""","""Ekstra""","""ORG""","""Kriminalitet""",196,null,0,null,0,0,510801,1494180,0,1686049852,13669,-1971510
284044691,1518744,9800148,7,0,1,false,0,0,0,false,41620968,false,10,1771,9112,33345,112,9,48.767857,55,76.590477,false,"""article_default""",142,"""sport""",238901,29148,1594379,0.8397,"""Neutral""","""Ange Postecoglou""","""PER""","""Erhverv""",196,null,0,null,0,0,510801,1494180,0,1686049852,5634,-1971511
284044691,1518744,9788021,7,0,1,false,0,0,0,false,41620968,false,10,2571,14480,95517,112,9,48.767857,55,76.590477,true,"""article_default""",140,"""krimi""",1437233,78228,4593527,0.9934,"""Negative""","""Daniel""","""PER""","""Kriminalitet""",0,null,0,null,0,0,510801,1494180,0,1686049852,21372,-1971500
284044691,1518744,9498042,7,0,1,false,0,0,0,false,41620968,false,10,136,766,4666,112,9,48.767857,55,76.590477,true,"""article_default""",457,"""forbrug""",0,0,0,0.9226,"""Neutral""",null,null,"""Økonomi""",472,null,0,null,0,0,510801,1494180,0,1686049852,18377646,-1971252
284044691,1518744,9799083,7,0,1,false,0,0,0,false,41620968,false,10,2418,14108,74332,112,9,48.767857,55,76.590477,false,"""article_scribblelive""",140,"""krimi""",517319,85599,5726493,0.9971,"""Negative""","""Louise Borglit""","""PER""","""Kriminalitet""",0,null,0,null,0,0,510801,1494180,0,1686049852,14267,-1971510


remove id from features....


In [8]:
sub = pd.concat(sub_list, ignore_index=True)
del sub_list
gc.collect()

0

## Output and Save

In [9]:
sub = pl.from_pandas(sub)
sub.write_parquet(f'{base_path}/output/score_lgbm_{train_type}_frac{TRAIN_FRAC}.parquet')
sub

impression_id,user_id,to_article_id,score
i32,i32,i32,f64
6451339,35982,9796527,-0.284338
6451339,35982,7851321,-0.854356
6451339,35982,9798805,0.130775
6451339,35982,9795150,-0.527686
6451339,35982,9531110,-0.621288
…,…,…,…
0,1225161,9792362,-0.610987
0,1225161,9788041,-0.626389
0,1225161,9790135,-0.637639


In [10]:
sub_sorted = sub.sort("impression_id", "user_id", "score", descending=[False, False, True])
sub_sorted = sub_sorted.with_columns(pl.col('score').rank(method="ordinal", descending=True).over(["impression_id", "user_id"]).alias('rank'))

out = sub.join(sub_sorted[['impression_id', 'user_id', 'to_article_id', 'rank']], on=['impression_id', 'user_id', 'to_article_id'], how='left')
out = out.group_by(['impression_id', 'user_id']).agg(pl.col('rank'))
id = pl.read_parquet(f"{out_path}/test_impression.parquet").select([
    pl.col("impression_id").cast(pl.Int32),
    pl.col("user_id").cast(pl.Int32)
])
out = id.join(out, on=["impression_id", "user_id"], how='left')
out

impression_id,user_id,rank
i32,i32,list[u32]
6451339,35982,"[4, 8, … 9]"
6451363,36012,"[1, 6, … 5]"
6451382,36162,"[4, 5, … 2]"
6451383,36162,"[1, 7, … 6]"
6451385,36162,"[4, 3, … 7]"
…,…,…
0,1589163,"[80, 164, … 176]"
0,1699456,"[68, 165, … 171]"
0,635479,"[91, 153, … 171]"


In [11]:
# check beyond accuracy impression
out.filter(pl.col('impression_id') == 0)

impression_id,user_id,rank
i32,i32,list[u32]
0,1049297,"[66, 140, … 189]"
0,231624,"[55, 149, … 189]"
0,716356,"[38, 158, … 195]"
0,1440307,"[68, 156, … 195]"
0,1822406,"[58, 156, … 176]"
…,…,…
0,1589163,"[80, 164, … 176]"
0,1699456,"[68, 165, … 171]"
0,635479,"[91, 153, … 171]"


In [12]:
with open(f'{base_path}/output/pred_lgbm_{train_type}_frac{TRAIN_FRAC}.txt', 'w') as f:
    for row in out.to_dicts():
        # 一度strにしないとlistの","の間に半角スペースが入ってしまう
        rank_str = ','.join(map(str, row['rank']))
        line = f"{row['impression_id']} [{rank_str}]\n"
        #line = f"{row['impression_id']} {row['rank']}\n"
        f.write(line)

### Check output

In [13]:
!head {base_path}/output/pred_lgbm_{train_type}_frac{TRAIN_FRAC}.txt

6451339 [4,8,3,5,7,2,1,6,9]
6451363 [1,6,3,7,8,2,4,5]
6451382 [4,5,1,3,2]
6451383 [1,7,11,3,4,5,10,2,8,9,6]
6451385 [4,3,2,6,5,1,7]
6451411 [9,1,8,3,2,6,7,5,4]
6451412 [7,4,1,5,8,6,2,3]
6451423 [33,26,30,3,17,31,11,14,24,28,13,10,18,32,9,27,25,8,20,15,7,5,12,16,4,21,29,22,23,6,2,19,1]
6451425 [2,3,1,5,6,4]
6451426 [2,5,1,3,4]


In [14]:
!tail {base_path}/output/pred_lgbm_{train_type}_frac{TRAIN_FRAC}.txt

0 [81,167,250,199,55,34,40,10,166,243,197,238,121,173,26,182,120,164,101,66,165,204,162,159,226,247,183,242,74,203,130,45,202,77,240,22,220,171,146,78,246,174,79,8,143,24,28,42,88,49,249,89,228,18,152,217,185,230,236,214,47,190,124,43,237,85,158,210,128,56,1,127,72,123,70,200,36,21,195,206,126,52,35,82,50,4,87,131,177,193,180,125,99,194,219,2,154,196,58,111,29,68,207,103,71,84,224,73,132,229,57,30,93,198,239,225,15,3,105,25,95,115,33,151,136,96,90,150,216,59,141,104,76,163,19,212,244,187,67,168,161,65,223,156,248,112,44,97,169,54,138,213,149,91,134,51,117,119,201,102,157,179,221,14,110,9,189,94,116,7,31,232,192,211,114,60,107,235,245,37,75,209,108,186,27,39,92,106,13,38,178,137,148,191,205,11,41,6,160,241,12,61,227,140,63,231,118,53,80,208,234,17,86,5,215,62,100,46,20,48,109,122,129,218,181,16,69,175,133,172,139,32,64,233,144,153,142,188,184,113,170,222,147,145,155,23,98,135,83,176]
0 [73,162,250,199,56,34,41,10,156,239,183,243,121,173,26,194,120,165,101,90,161,204,159,158,226,247,196,

In [15]:
!wc -l {base_path}/output/pred_lgbm_{train_type}_frac{TRAIN_FRAC}.txt

13536710 /content/drive/MyDrive/RecSys2024//output/pred_lgbm_train_frac1.0.txt
